# This notebook implements the BOW Classification Model

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import glob
from importlib import reload

import os, sys, re, json, time, datetime, shutil
from common import utils, constants, spell

import tensorflow as tf
import tripadvisor_ds
import visualization

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

## 1. Load tripadvisor data¶

In [2]:

reload(tripadvisor_ds)

input_length = 500
max_bytes = 2**31 - 1

data_file = 'data/tripadvisor_ds.pkl'

if os.path.isfile(data_file):

    bytes_in = bytearray(0)
    input_size = os.path.getsize(data_file)
    with open(data_file, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    ds = pickle.loads(bytes_in)
        
else:
    ds = tripadvisor_ds.TripAdvisor_DS().process(input_length=input_length)
    ds.save(data_file)


In [5]:
labels = pd.DataFrame({'rating': ds.train_labels})
labels.rating.value_counts()

3    413463
2    231827
1    106079
0     90053
Name: rating, dtype: int64

## 2. Build the Model

In [7]:
# set model parameters 
model_params = dict(V=ds.vocab.size, 
                    embed_dim=100, 
                    num_classes=len(ds.target_labels),
                    encoder_type='bow', 
                    hidden_dims=[1024, 64], 
                    input_length=input_length,
                    lr=0.0001, 
                    optimizer='adam', 
                    beta=0.0001)
                    
train_params = dict(batch_size=64, 
                    total_epochs=20, 
                    eval_every=2)


summary_params = dict(chkpt_dir="./tmp/266_cnn_" + datetime.datetime.now().strftime("%Y%m%d-%H%M"))

In [9]:
# setup tensorboard 

if os.path.isdir(summary_params['chkpt_dir']):
    shutil.rmtree(summary_params['chkpt_dir'])

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [10]:
def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable("W_embed", shape=[V, embed_dim], \
                               initializer=tf.random_uniform_initializer(-init_scale, init_scale), \
                               trainable=True)
    
    #W_embed_ = tf.constant(embedding_matrix, dtype=tf.float32, name="W_embed")
    
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_, name="embed_x")
        
    return xs_

def fully_connected_layers(h0_, hidden_dims, activation=tf.nn.relu,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        h_ = tf.layers.dense(h_, hdim, activation=activation, name=("Hidden_%d"%i))
        if dropout_rate > 0:
            #h_ = tf.layers.dropout(h_, rate=dropout_rate, training=is_training )
            h_ = tf.layers.dropout(h_, training=is_training )
    return h_

def softmax_output_layer(h_, labels_, num_classes):
    
    W_out_ = tf.get_variable("W_out",  shape=[h_.get_shape().as_list()[1], num_classes], \
                               initializer=tf.random_normal_initializer())
    b_out_ = tf.get_variable("b_out", shape=[num_classes])

    logits_ = tf.add(tf.matmul(h_, W_out_), b_out_)
        
    if labels_ is None:
        return None, logits_
    
    with tf.variable_scope("Softmax_Layer"):

        softmax_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_, logits=logits_)
        
        loss_ = tf.reduce_mean(softmax_)
    
    return loss_, logits_

def BOW(ids_, V, embed_dim, hidden_dims, dropout_rate=0, is_training=None):
    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)
     
    sum_xs_ = tf.reduce_sum(xs_, 1)

    h_ = fully_connected_layers(sum_xs_, hidden_dims, \
                           dropout_rate=dropout_rate, is_training=is_training)
    return h_, xs_


def conv_net(ids_, V, embed_dim, filter_sizes, num_filters, hidden_dims, input_length, dropout_rate=0, is_training=None):

    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)

    xs_ = tf.expand_dims(xs_, -1)
        
    pooled_outputs_ = []
    for _, filter_size in enumerate(filter_sizes):
        with tf.name_scope("Conv_MaxPool_%d"%filter_size):
            
            # Convolution Layer
            filter_shape = [filter_size, embed_dim, 1, num_filters]
            W_ = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b_ = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv_ = tf.nn.conv2d(
                xs_,
                W_,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            
            # Activation
            h_ = tf.nn.relu(tf.nn.bias_add(conv_, b_), name="relu")
            
            # Maxpooling 
            pooled_ = tf.nn.max_pool(
                h_,
                ksize=[1, input_length - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
            pooled_outputs_.append(pooled_)
            
            variable_summaries(pooled_)

    # Combine all the pooled features and flatten it
    num_filters_total = num_filters * len(filter_sizes)
    h_ = tf.concat(pooled_outputs_, 3)
    h_ = tf.reshape(h_, [-1, num_filters_total])
    
    # fully connected layers
    with tf.variable_scope("FC_Layer"):
        h_ = fully_connected_layers(h_, hidden_dims, is_training = is_training)

    return h_, xs_


In [11]:
tf.reset_default_graph() 

X = tf.placeholder(tf.int32, [None, input_length], name='input_x')
Y = tf.placeholder(tf.int32, [None,], name='input_y')
    

if Y is None:
    one_hot_y_ = None
else:
    one_hot_y_ = tf.one_hot(Y, model_params['num_classes'])


if model_params['encoder_type'] == 'cnn':
    h_, xs_ = conv_net(X, model_params['V'], 
                      model_params['embed_dim'], 
                      model_params['filter_sizes'], 
                      model_params['num_filters'], 
                      model_params['hidden_dims'],
                      model_params['input_length'],
                      is_training=True)
elif model_params['encoder_type'] == 'bow':
    h_, xs_ = BOW(X, model_params['V'], 
                      model_params['embed_dim'],  
                      model_params['hidden_dims'],
                      is_training=True)


with tf.variable_scope("Output_Layer"):
    ce_loss_, logits_ = softmax_output_layer(h_, Y, model_params['num_classes'])
    

with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
    pred_max_ = tf.argmax(logits_, 1, name="pred_max")
    predictions_dict = {"proba": pred_proba_, "max": pred_max_}

with tf.variable_scope("Regularization"):
    l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
    for var_ in tf.trainable_variables():
        if "Embedding_Layer" in var_.name:
            continue
        l2_penalty_ += tf.nn.l2_loss(var_)
    l2_penalty_ *= model_params['beta']  # scale by regularization strength
    tf.summary.scalar('l2_penalty', l2_penalty_)
    regularized_loss_ = ce_loss_ + l2_penalty_
    tf.summary.scalar('regularized_loss', regularized_loss_)

with tf.variable_scope("Training"):
    if model_params['optimizer'] == 'adagrad':
        optimizer_ = tf.train.AdagradOptimizer(model_params['lr'])
    elif  model_params['optimizer'] == 'adam':
        optimizer_ = tf.train.AdamOptimizer(model_params['lr'])
    else:
        optimizer_ = tf.train.GradientDescentOptimizer(model_params['lr'])
    train_op_ = optimizer_.minimize(regularized_loss_,
                    global_step=tf.train.get_global_step())


with tf.name_scope("Evaluation"):
    cross_entropy_loss_ = tf.metrics.mean(ce_loss_)
    
    correct_pred_ = tf.equal(tf.argmax(logits_, 1), tf.argmax(one_hot_y_, 1))
    accuracy_ = tf.reduce_mean(tf.cast(correct_pred_, tf.float32))

    eval_metrics = {"cross_entropy_loss": cross_entropy_loss_, "accuracy": accuracy_}
    
    tf.summary.scalar('cross_entropy', ce_loss_)
    tf.summary.scalar('accuracy', accuracy_)



## 3. Train the Model

In [ ]:


# start session
sess = tf.Session()
graph = tf.get_default_graph()

# Tensorboard - Visualize graph 
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/train', sess.graph)
test_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/test')

print("tensorboard --logdir={}/train".format(summary_params['chkpt_dir']))
print("tensorboard --logdir={}/test".format(summary_params['chkpt_dir']))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

# Run the initializer
sess.run(init)
sess.run(init_l)

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)



for i in range(train_params['total_epochs']):
    t0 = time.time()

    train_batches = 1
    train_accuracy = 0.0
    
    for (bx, by) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_train_features, ds.train_labels):

        summary, batch_loss, _, batch_accuracy = sess.run(
            [merged, regularized_loss_, train_op_, accuracy_], feed_dict={X: bx, Y: by})
        
        train_batches +=1
        train_accuracy += batch_accuracy
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean

        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 25 == 0):
            print("{:5,} examples, moving-average loss {:.2f}, train accuracy {:.2f}"\
                  .format(total_examples, loss_ema, train_accuracy/train_batches))    
            
        train_writer.add_summary(summary, total_batches)

    print("Completed {} epoch in {:s}".format(i, utils.pretty_timedelta(since=t0)))
    
    train_accuracy = train_accuracy/train_batches
    print("Train accurary:{:.5f}".format(train_accuracy))
    
    
    # run the validation dataset 
    validate_batches = 1
    validate_accuracy = 0.0
    for (vx, vy) in utils.multi_batch_generator(train_params['batch_size'], \
                                            ds.padded_validate_features, ds.validate_labels):

        summary, batch_accuracy = sess.run([merged, accuracy_], feed_dict={X: vx, Y: vy})

        validate_batches +=1
        validate_accuracy += batch_accuracy

        test_writer.add_summary(summary, total_batches + validate_batches)

    validate_accuracy = validate_accuracy/validate_batches
    print("Validate accuracy:{:.5f}".format(validate_accuracy))
        

## 4. Evaluate the Model 

In [18]:
test_batches = 1
test_accuracy = 0.0
test_pred_y = []

for (tx, ty) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_test_features, ds.test_labels):

    batch_accuracy, pred_max = sess.run([accuracy_, pred_max_], feed_dict={X: tx, Y: ty})

    test_batches +=1
    test_accuracy += batch_accuracy
    test_pred_y.append(pred_max.tolist())
    
test_accuracy = test_accuracy/test_batches
print("Test accuracy:{}".format(test_accuracy))

pred_y = [y for x in test_pred_y for y in x]

Test accuracy:0.7110148514851485


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ds.test_labels, pred_y)

array([[ 8244,  2245,   367,   100],
       [ 1902,  6760,  3952,   417],
       [  256,  2283, 16288,  9850],
       [   69,   261,  7482, 43403]])

In [23]:
#op = sess.graph.get_operations()
#[m.values() for m in op]



## 5. Implement Integrated Gradients 

In [19]:
def get_gradients(sess, graph, review):
    
    with graph.as_default():

        # get prediction 
        pred_prob_ = graph.get_tensor_by_name('Prediction/pred_proba:0')
        pred_prob = sess.run([pred_prob_], feed_dict={X:[review], Y:[0]})
        
        # get gradient     
        input_y_ = graph.get_tensor_by_name('input_y:0')[0]
        embed_x_ = graph.get_tensor_by_name('Embedding_Layer/embed_x:0')
        label_prediction_ = graph.get_tensor_by_name('Prediction/pred_proba:0')[:, input_y_]
        grads_ = tf.gradients(label_prediction_, embed_x_)[0]
        
        embed_x, grads = sess.run([embed_x_, grads_], feed_dict={X:[review], Y:[np.argmax(pred_prob)]})
        return grads * embed_x


In [20]:
def get_integrated_gradients_new(sess, graph, review, steps = 50):

    empty_review = np.zeros(input_length)
    
    with graph.as_default():

        # get embedding and prediction 
        embed_x_ = graph.get_tensor_by_name('Embedding_Layer/embed_x:0')
        pred_prob_ = graph.get_tensor_by_name('Prediction/pred_proba:0')
        embed_x, pred_y, pred_prob = sess.run([embed_x_, pred_max_, pred_prob_], feed_dict={X:[review], Y:[4]})
          
        # empty embedding 
        empty_embed_x, empty_pred_prob = sess.run([embed_x_, pred_prob_], feed_dict={X:[empty_review], Y:[4]})
            
            
        # get integrated gradient 
        input_y_ = graph.get_tensor_by_name('input_y:0')[0]
        label_prediction_ = graph.get_tensor_by_name('Prediction/pred_proba:0')[:, input_y_]
        grads_ = tf.gradients(label_prediction_, embed_x_)[0]

        
        all_grads = []
        for i in range(0, steps + 1):
            _, grads = sess.run([label_prediction_, grads_], feed_dict={embed_x_:(empty_embed_x + (embed_x - empty_embed_x)*(float(i)/steps)), Y:pred_y})
            all_grads.append(grads)
        
        integrated_grads = np.average(all_grads[:-1], axis=0) * embed_x
        return integrated_grads, pred_y[0], pred_prob, empty_pred_prob
        

### 5.1 supporing classes to save the output of the Integrated Gradients 

In [26]:
class AttributionResult(object):
    def __init__(self, filename):
        self.filename = filename
        self.html_text = "<html><body>"
    
    def append(self, text):
        self.html_text += text
    
    def write(self, index):
        self.html_text += "</body></html>"
        
        with open("result/{}_{}.html".format(self.filename, index), "w") as the_file:
            the_file.write(self.html_text)
            
        self.html_text = "<html><body>"
        
class AttributionCount(object):
    def __init__(self, filename):
        self.filename = filename
        self.word_count = []
        
    def append(self, token):
        self.word_count.append(token)
    
    def write(self, index):
        df = pd.DataFrame.from_records(self.word_count)
        df.columns = ['pred', 'target', 'vocab', 'attr']
        df.attr.round(5)
        df.to_csv("result/{}_{}.csv".format(self.filename, index))
    
        self.word_count = []
        

### 5.2 Visualize attributions 

In [ ]:
reload(visualization)

# save the attributions 
file_suffix = "bow_d{}_h1024_64_epoch{}".format(model_params['embed_dim'], \
                                   train_params['total_epochs'])

corrHigh = AttributionResult("CorrHigh_{}".format(file_suffix))
corrLow = AttributionResult("CorrLow_{}".format(file_suffix))
incorrHigh = AttributionResult("InCorrHigh_{}".format(file_suffix))
incorrLow = AttributionResult("InCorrLow_{}".format(file_suffix))
word_counts = AttributionCount("wordcounts_{}".format(file_suffix))


# loop through the test data to visualize the result 

for i in range(0, len(ds.test_labels)):
    gradients = get_gradients(sess, sess.graph, ds.padded_test_features[i])
    integrated_gradients, pred_y, pred_proba, empty_pred_proba = \
                                get_integrated_gradients_new(sess, sess.graph, ds.padded_test_features[i])
   
    
    #print(np.matrix(gradients).sum())
    print(np.matrix(integrated_gradients).sum())
    print(pred_proba[0][pred_y])
    print(empty_pred_proba[0][pred_y])

    pred_text = "predicted = {}, actual = {}".format(pred_y+2, ds.test_labels[i]+2)
    print("{}, {}".format(i, pred_text))

    html_text = visualization.visualize_token_attrs(
            ds.vocab.ids_to_words(ds.test_features[i])[:input_length], 
            np.matrix(integrated_gradients).sum(axis=1), 
            pred_y+2, ds.test_labels[i]+2, word_counts)

    text = "</br>" + pred_text + "</br>" + html_text
    
    if (pred_y == ds.test_labels[i]):
        if (pred_y >= 2):
            corrHigh.append(text)
        else:
            corrLow.append(text)
    else:
        if (ds.test_labels[i] >= 2):
            incorrHigh.append(text)
        else:
            incorrLow.append(text)
    
    if (i % 1000) == 0:
        corrHigh.write(i)
        corrLow.write(i)
        incorrHigh.write(i)
        incorrLow.write(i)
        
        word_counts.write(i)

# 6. Sensitivity Analysis 

In [ ]:
from nltk import word_tokenize

#prefix = "jeff go to supermarket and buy oranges for dinner: "
#prefix = "we get on a taxi and go to times square in the morning: "
#prefix = "we were in NYC last week and stay at Hilton.  The service there was great. : "
#prefix = "we visited my mom and she was not there and we end up going to spa : "
prefix = "%%%%%%% : "


prefix_tokens = word_tokenize(prefix.lower())

# Function to add prefix to input features 
def add_prefix(features):   
    new_features = ds.vocab.words_to_ids(utils.canonicalize_word(w, ds.vocab) for w in prefix_tokens)
    new_features.extend(features)
    return new_features[0:input_length]

    
sensitivity_batches = 100  
sensitivity_sample_size = train_params['batch_size'] * sensitivity_batches

# ---------------------------------------------------------
# check the accuracy of the sample before adding the prefix 
# ---------------------------------------------------------
test_batches = 1
test_accuracy = 0.0
test_pred_y = []

for (tx, ty) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_test_features, ds.test_labels):

    batch_accuracy, pred_max = sess.run([accuracy_, pred_max_], feed_dict={X: tx, Y: ty})

    test_batches +=1
    test_accuracy += batch_accuracy
    test_pred_y.append(pred_max.tolist())
    
    if test_batches == sensitivity_batches:
        break
    
test_accuracy = test_accuracy/test_batches
print("Sensitivity test accuracy before adding prefix:{}".format(test_accuracy))

    
# ---------------------------------------------------------
# check the accuracy of the sample before adding the prefix 
# ---------------------------------------------------------
test_batches = 1
test_accuracy = 0.0
test_pred_y = []

# Add prefix to the user reviews

sensitivity_features = []
sensitivity_labels = ds.test_labels[:sensitivity_sample_size]
for i in range(sensitivity_sample_size):
    sensitivity_features.append(add_prefix(ds.padded_test_features[i]))

for (tx, ty) in utils.multi_batch_generator(train_params['batch_size'], \
                                        sensitivity_features, sensitivity_labels):

    batch_accuracy, pred_max = sess.run([accuracy_, pred_max_], feed_dict={X: tx, Y: ty})

    test_batches +=1
    test_accuracy += batch_accuracy
    test_pred_y.append(pred_max.tolist())
    

test_accuracy = test_accuracy/test_batches
print("Sensitivity test accuracy after adding prefix:{}".format(test_accuracy))


# --------------------------------------------------------------
# Visualize the attributions on user reviewers with added prefix 
# --------------------------------------------------------------

for i in range(0, 10):
    
    integrated_gradients, pred_y, pred_proba, empty_pred_proba = \
                                get_integrated_gradients_new(sess, sess.graph, sensitivity_features[i])
   
    print(np.matrix(integrated_gradients).sum())
    print(pred_proba[0][pred_y])
    print(empty_pred_proba[0][pred_y])

    pred_text = "predicted = {}, actual = {}".format(pred_y+2, sensitivity_labels[i]+2)
    print("{}, {}".format(i, pred_text))

    #visualization.visualize_token_attrs(ds.vocab.ids_to_words(ds.test_features[i])[:input_length], np.matrix(gradients).sum(axis=1))
    html_text = visualization.visualize_token_attrs(
            ds.vocab.ids_to_words(sensitivity_features[i])[:input_length], 
            np.matrix(integrated_gradients).sum(axis=1), 
            pred_y+2, sensitivity_labels[i]+2)

    text = "</br>" + pred_text + "</br>" + html_text
 
